# BioPython Tutorial

[Biopython](https://biopython.org/) is a module with a variety of functions that are useful for working with sequence data. We will use the following examples to help you make use of sequence data. 

## Using BioPython to examine sequence files

Your sequence data are in the `.ab1` format. This format is common for Sanger sequencing data. First, let's take a look at the files:

In [ ]:
!ls data/

Let's just choose a single file `Student_1-M13F.ab1` to open and explore with BioPython: 

In [ ]:
# import SeqIO from the Bio library/module
# SeqIO lets us handle the file (open and read from it)
from Bio import SeqIO
 
# specify the path of the file
file_path = "data/sample_sequence-M13F.ab1"
 
# We create an "object" using the SeqIO.read function 
sequence_object = SeqIO.read(open(file_path,"rb"),"abi")  

#Next, use the .format method to extract the DNA sequence in FASTA format
fasta_sequence = sequence_object.format("fasta")
print(fasta_sequence) 

# What other methods are associated with the sequence object?
# See a full list at https://biopython.org/wiki/SeqIO

We can also print the sequence quality scores...

In [ ]:
# get the quality scores for each base
quals = sequence_object.letter_annotations['phred_quality']
print(quals)

## Aligning your forward and reverse sequences

Let's open the reverse sequence read (that hopefully also worked):

In [ ]:

# specify the path of the file (make sure to choose the sequence with the "M13R" primer)
file_path = "data/sample_sequence-M13R.ab1"
 
# We create an "object" using the SeqIO.read function 
sequence_object = SeqIO.read(open(file_path,"rb"),"abi")  

#Next, use the .format method to extract the DNA sequence in FASTA format
fasta_sequence_reverse = sequence_object.format("fasta")


#We should now have both sequences...

print(fasta_sequence) 

print(fasta_sequence_reverse) 

Well... not quite, these are Fasta sequences, but what we really need in order to use other methods is the sequence object, let's get those. Also, for convenience lets rename the original sequence to indicate it is the forward read:



In [ ]:
file_path_1 = "data/sample_sequence-M13F.ab1"
foward_sequence_object = SeqIO.read(open(file_path_1,"rb"),"abi")  

file_path_2 = "data/sample_sequence-M13R.ab1"
reverse_sequence_object = SeqIO.read(open(file_path_2,"rb"),"abi") 

We also need to reverse compliment the reverse read:

In [ ]:
from Bio.Seq import Seq
from Bio.Alphabet import generic_dna

reverse_sequence_object_rc = reverse_sequence_object.reverse_complement(description="reverse sequence")

print(reverse_sequence_object.format("fasta"))
print(reverse_sequence_object_rc.format("fasta")) # just proving that we have reverse complimented

We can also print the forward and reverse complimented sequences and start to see some similarities

In [ ]:
print(foward_sequence_object.format("fasta"))
print(reverse_sequence_object_rc.format("fasta")) 

First, we will write our sequence objects to a single file

In [ ]:
#write the dna to file
with open('input.fa','w') as result_file:
    result_file.write(foward_sequence_object.format("fasta"))
    result_file.write(reverse_sequence_object_rc.format("fasta"))

Then we can do an alignment using a software called [MUSCLE](https://www.drive5.com/muscle/). 

In [ ]:
# Download and install MUSCLE 

# Linux 
#!wget https://www.drive5.com/muscle/downloads3.8.31/muscle3.8.31_i86linux64.tar.gz && tar -xvf muscle3.8.31_i86linux64.tar.gz
# Mac: 
#!wget https://www.drive5.com/muscle/downloads3.8.31/muscle3.8.31_i86darwin64.tar.gz && tar -xvf muscle3.8.31_i86darwin64.tar.gz

Now we will formulate the MUSCLE command

In [ ]:
# Linux 
#!./muscle3.8.31_i86linux64 -in input.fa -out alignment.txt -clw
#Mac
#!./muscle3.8.31_i86darwin64 -in input.fa -out alignment.txt -clw

And we can view the alignment:


In [ ]:
!cat alignment.txt

## Plotting sequence quality

We can plot our quality score as a histogram:

In [ ]:
# Since quals is a list, what would it look like to plot this...

import numpy as np
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
% matplotlib inline

plot = plt.hist(quals, facecolor='blue', alpha=0.75)

plt.xlabel('Phred Scores')
plt.ylabel('Number of nucleotides')
plt.title('Histogram of Phred scores')

plt.show()

In [ ]:
import numpy as np                                                               
import matplotlib.pyplot as plt
% matplotlib inline

xs = np.arange(len(quals)) 
width = 10

plt.plot(xs, quals)

plt.show()

In [ ]:
channels = ['DATA9', 'DATA10', 'DATA11', 'DATA12']
from collections import defaultdict
trace = defaultdict(list)
for c in channels:
    trace[c] = sequence_object.annotations['abif_raw'][c]

plt.plot(trace['DATA9'][::10], color='blue')
plt.plot(trace['DATA10'][::10], color='red')
plt.plot(trace['DATA11'][::10], color='green')
plt.plot(trace['DATA12'][::10], color='yellow')

plt.show()

## Blasting a sequence

Next, let's search NCBI to see what matches we can return from our sequence. This may take up to a few minutes

In [ ]:
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML

#blast the sequence result using blastn, against the
#nr database, evalue of 0.0001, return top 5 hits
blast_result = NCBIWWW.qblast('blastn','nt',fasta_sequence, expect = 0.0001, hitlist_size=5) 

blast_record = NCBIXML.read(blast_result)

# print the blast hit results

for records in blast_record.alignments:
    print(records)

# Where to go next

See a whole list of BioPython tutorials that you can use in putting together your own notebook...http://biopython.org/DIST/docs/tutorial/Tutorial.html
